In [9]:
import numpy as np
import cv2
path = "/home/yifan/MyCode/calibration/MC-Calib/data/Real_Images/Seq01_Non-overlapping/calibrated_cameras_data.yml"
# 读取 YAML 文件
fs = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)
nb_camera = int(fs.getNode("nb_camera").real())
print("nb_camera: ", nb_camera)
camera_data = {}
for i in range(nb_camera):
    camera_data[i] = {}
    data = fs.getNode("camera_" + str(i))
    intrinsic_matrix = data.getNode("camera_matrix").mat()
    intrinsic_vector = [intrinsic_matrix[0, 0], intrinsic_matrix[1, 1], intrinsic_matrix[0, 2], intrinsic_matrix[1, 2]]
    distortion_coefficients = data.getNode("distortion_vector").mat()
    distortion_type = int(data.getNode("distortion_type").real())
    img_width = int(data.getNode("img_width").real())
    img_height = int(data.getNode("img_height").real())
    camera_pose_matrix = data.getNode("camera_pose_matrix").mat()
    camera_data[i]["intrinsic_vector"] = intrinsic_vector
    camera_data[i]["distortion_coefficients"] = distortion_coefficients
    camera_data[i]["distortion_type"] = distortion_type
    camera_data[i]["img_width"] = img_width
    camera_data[i]["img_height"] = img_height
    camera_data[i]["camera_pose_matrix"] = camera_pose_matrix
    print("{} cam's camera_pose_matrix: ".format(i), camera_pose_matrix)
    print("{} cam's intrinsic_vector: ".format(i), intrinsic_vector)
    print("{} cam's distortion_coefficients: ".format(i), distortion_coefficients)
    print("{} cam's distortion_type: ".format(i), distortion_type)
    print("{} cam's img_width: ".format(i), img_width)
    print("{} cam's img_height: ".format(i), img_height)

nb_camera:  4
0 cam's camera_pose_matrix:  [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
0 cam's intrinsic_vector:  [229.85069502005703, 229.83164706378273, 298.1905725488463, 225.90148913924767]
0 cam's distortion_coefficients:  [[ 0.02119024 -0.0308355   0.04019453 -0.01857033]]
0 cam's distortion_type:  1
0 cam's img_width:  640
0 cam's img_height:  480
1 cam's camera_pose_matrix:  [[ 0.99986554  0.00605422 -0.01524     0.74165941]
 [-0.00614086  0.99996522 -0.00564422 -0.00530152]
 [ 0.0152053   0.00573705  0.99986793  0.00688099]
 [ 0.          0.          0.          1.        ]]
1 cam's intrinsic_vector:  [231.16637798892714, 230.96296583754366, 300.71186424950207, 232.02638267225933]
1 cam's distortion_coefficients:  [[ 0.01925885 -0.01882203  0.0231466  -0.01046778]]
1 cam's distortion_type:  1
1 cam's img_width:  640
1 cam's img_height:  480
2 cam's camera_pose_matrix:  [[-0.99984616 -0.01039522  0.01412751  0.00950633]
 [-0.01019388  0.9998465   0.01424986  0.

In [13]:
import yaml
import numpy as np
# 将数据写入 YAML 文件，格式类似上述格式（kalibr）
overlaps_map = {0:1, 1:0, 2:3, 3:2}
cam_kalibr_data = {}
cam_kalibr_data["cam0"] = {}
cam_kalibr_data["cam0"]["cam_overlaps"] = [overlaps_map[0]]
cam_kalibr_data["cam0"]["camera_model"] = "pinhole"
cam_kalibr_data["cam0"]["distortion_coeffs"] = list(camera_data[0]["distortion_coefficients"].tolist())
cam_kalibr_data["cam0"]["distortion_model"] = "radtan"
cam_kalibr_data["cam0"]["intrinsics"] = np.array(camera_data[0]["intrinsic_vector"]).tolist()
cam_kalibr_data["cam0"]["resolution"] = [camera_data[0]["img_width"], camera_data[0]["img_height"]]
cam_kalibr_data["cam0"]["rostopic"] = "/cam0/image_raw"
for i in range(1, nb_camera):
    cam_str = "cam" + str(i)
    print(cam_str)
    cam_kalibr_data[cam_str] = {}
    cam_kalibr_data[cam_str]["T_cn_cnm1"] = np.dot(np.linalg.inv(camera_data[i]["camera_pose_matrix"]), camera_data[i-1]["camera_pose_matrix"]).tolist()
    cam_kalibr_data[cam_str]["cam_overlaps"] = [overlaps_map[i]]
    cam_kalibr_data[cam_str]["camera_model"] = "pinhole"
    cam_kalibr_data[cam_str]["distortion_coeffs"] = camera_data[i]["distortion_coefficients"].tolist()
    cam_kalibr_data[cam_str]["distortion_model"] = "radtan"
    cam_kalibr_data[cam_str]["intrinsics"] = np.array(camera_data[i]["intrinsic_vector"]).tolist()
    cam_kalibr_data[cam_str]["resolution"] = [camera_data[i]["img_width"], camera_data[i]["img_height"]]
    cam_kalibr_data[cam_str]["rostopic"] = "/cam" + str(i) + "/image_raw"

class MyDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(MyDumper, self).increase_indent(flow, False)

    def represent_dict(self, data):
        value = []
        node = yaml.nodes.MappingNode(tag='tag:yaml.org,2002:map', value=value)
        if self.alias_key is not None:
            self.represented_objects[self.alias_key] = node
        best_style = True
        for item_key, item_value in data.items():
            node_key = self.represent_data(item_key)
            node_value = self.represent_data(item_value)
            if not isinstance(node_key, yaml.nodes.ScalarNode) or not isinstance(node_value, yaml.nodes.ScalarNode):
                best_style = False
            value.append((node_key, node_value))
        if self.default_flow_style is None:
            node.flow_style = best_style
        else:
            node.flow_style = self.default_flow_style
        return node

file_path = "/mnt/dataset/calibration/Mc-test/overlapping/images/2024-06-18-17-37-56_1844c160c6ee1c84/cam_imu-camchain.yaml"

with open(file_path, 'w') as f:
    yaml.dump(cam_kalibr_data, f, Dumper=MyDumper, default_flow_style=None)

    

cam1
cam2
cam3


In [15]:
import numpy as np
import yaml
path = "/mnt/dataset/calibration/cam_imu-camchain-imucam.yaml"
# 读取 YAML 文件
data = yaml.safe_load(open(path, 'r'))

In [18]:
T_cam_imu_0 = np.array(data['cam0']['T_cam_imu'])
T_cam_imu_0

array([[ 0.11550656, -0.9923333 , -0.04396435,  0.06434952],
       [ 0.99300129,  0.11645541, -0.01966172,  0.02807321],
       [ 0.02463086, -0.0413856 ,  0.9988396 ,  0.16779004],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
T_cam_imu_1 = np.array(data['cam1']['T_cam_imu'])
T_cam_imu_1

array([[ 0.08727932, -0.99617209, -0.00484627, -0.00979779],
       [ 0.99617448,  0.0872986 , -0.00391999,  0.03234276],
       [ 0.00432806, -0.0044856 ,  0.99998057,  0.16333762],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
np.dot( T_cam_imu_1, np.linalg.inv(T_cam_imu_0))

array([[ 9.98829131e-01, -2.92458622e-02,  3.85362963e-02,
        -7.97169494e-02],
       [ 2.86077195e-02,  9.99446005e-01,  1.70082892e-02,
        -4.09615686e-04],
       [-3.90123695e-02, -1.58859392e-02,  9.99112442e-01,
        -1.34709791e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])